In [1]:
!pip install langchain langchain_community langchain-google-genai python-dotenv langchain_experimental langchain_chroma langchainhub pypdf

In [2]:
from langchain_community.document_loaders import PyPDFLoader

# Update the path to your PDF file
loader = PyPDFLoader("/content/Cust_sup.pdf")
data = loader.load()  # entire PDF is loaded as a single Document

# Verify the data
print(data)


[Document(metadata={'source': '/content/Cust_sup.pdf', 'page': 0, 'page_label': '1'}, page_content='Customer Support Ticket Examples \nTicket 1: Password Reset Request \n• Customer Query: "I forgot my password and can\'t log in to my account. Can you help me \nreset it?" \n• Response: "Dear [Customer Name], we have received your request to reset your password. \nPlease click on the following link to reset your password: [Reset Link]. If you encounter any \nissues, feel free to contact us. Thank you!" \nTicket 2: Billing Issue \n• Customer Query: "I was charged twice for my subscription this month. Can you please look \ninto this?" \n• Response: "Dear [Customer Name], we apologize for the inconvenience caused by the \ndouble charge. Our billing team is currently investigating the issue. You will receive a refund \nfor the extra charge within 3-5 business days. Thank you for your patience. " \nTicket 3: Product Defect \n• Customer Query: "The product I received is defective. It doesn\'t 

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  20


In [4]:
import os
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Load environment variables from a .env file
load_dotenv()
os.environ["GOOGLE_API_KEY"] = api_key

# Initialize the embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
print(vector[:5])


[0.05168594419956207, -0.030764883384108543, -0.03062233328819275, -0.02802734263241291, 0.01813093200325966]


In [5]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [32]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("i want to reset password ?")

In [33]:
len(retrieved_docs)

3

In [34]:
for i in range(0,len(retrieved_docs)):
  print(retrieved_docs[i].page_content)

• Response: "Dear [Customer Name], if you forget your password, you can recover it by 
clicking on the 'Forgot Password' link on the login page. You will be prompted to enter your 
registered email address, and a password reset link will be sent to you. If you encounter any 
issues, please contact our support team. "
Ticket 13: Password Reset Link Not Working 
• Customer Query: "I tried using the password reset link, but it's not working. What should I 
do?" 
• Response: "Dear [Customer Name], we apologize for the inconvenience caused by the 
non-functional reset link. Please try again using the following link: [New Reset Link]. If the 
issue persists, contact our support team for further assistance. " 
Ticket 14: Password Change Confirmation
support team with your account details, and we will assist you further. " 
Additional Password-Related Tickets 
Ticket 11: Password Expiry Notification 
• Customer Query: "I received a notification that my password is about to expire. How can I 
u

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [11]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "I want to reset my password ?"})
print("RAG Output:", response["answer"])

RAG Output: Click the "Forgot Password" link on the login page.  You'll be prompted to enter your registered email address to receive a password reset link. Contact support if you have any issues.
